In [1]:
import sys
sys.path.append("../input/segmentation-models-pytorch/segmentation_models.pytorch-0.2.1/")
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append("../input/pretrainedmodels/pretrainedmodels-0.7.4")
sys.path.append("../input/effnet-pytorch/efficientnet_pytorch-0.6.3/")
sys.path.append("../input/pytorch-toolbelt/pytorch-toolbelt-develop/")
import pandas as pd
import numpy as np
import os
from glob import glob
import gc
gc.enable()
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from PIL import Image
import cv2
import albumentations as A
import time
import os
from tqdm import tqdm
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset
from typing import List
from pytorch_toolbelt.inference import (
    GeneralizedTTA,
    d4_image_augment,
    d4_image_deaugment,
)
from os.path import join as path_join
from skimage.filters import sobel, meijering
from typing import Optional

In [2]:
!pip uninstall -y pytorch_lightning
sys.path.append('../input/lightning-163/pytorch_lightning/')
sys.path.append('../input/lightning-163/')
import pytorch_lightning as pl

Found existing installation: pytorch-lightning 1.5.10
Uninstalling pytorch-lightning-1.5.10:
  Successfully uninstalled pytorch-lightning-1.5.10


In [3]:
df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv")
DEBUG = False
if df.shape[0] == 0:
    print('Using part of train data as example')
    DEBUG = True
if DEBUG == True:
    df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/train.csv")
    df.pop('segmentation')
    df['predicted'] = ""
df.head()

Using part of train data as example


,id,class,predicted
0,case123_day20_slice_0001,large_bowel,
1,case123_day20_slice_0001,small_bowel,
2,case123_day20_slice_0001,stomach,
3,case123_day20_slice_0002,large_bowel,
4,case123_day20_slice_0002,small_bowel,


In [4]:
df["case_id_str"] = df["id"].apply(lambda x: x.split("_", 2)[0])
df["case_id"] = df["id"].apply(lambda x: int(x.split("_", 2)[0].replace("case", "")))

# 2. Get Day as a column
df["day_num_str"] = df["id"].apply(lambda x: x.split("_", 2)[1])
df["day_num"] = df["id"].apply(lambda x: int(x.split("_", 2)[1].replace("day", "")))

# 3. Get Slice Identifier as a column
df["slice_id"] = df["id"].apply(lambda x: x.split("_", 2)[2])

if DEBUG:
    TRAIN_DIR = '../input/uw-madison-gi-tract-image-segmentation/train'
else:
    TRAIN_DIR = '../input/uw-madison-gi-tract-image-segmentation/test'
# Get all training images
all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive=True)
""
p = []
x = all_train_images[0].rsplit("/", 4)[0]
for i in range(0, df.shape[0]):
    p.append(os.path.join(x, df["case_id_str"].values[i], df["case_id_str"].values[i]+"_"+df["day_num_str"].values[i], "scans", df["slice_id"].values[i]))
df["_partial_ident"] = p

p = []
for i in range(0, len(all_train_images)):
    p.append(str(all_train_images[i].rsplit("_",4)[0]))
    
_tmp_merge_df = pd.DataFrame()
_tmp_merge_df['_partial_ident'] = p
_tmp_merge_df['f_path'] = all_train_images


df = df.merge(_tmp_merge_df, on="_partial_ident").drop(columns=["_partial_ident"])

# 5. Get slice dimensions from filepath (int in pixels)
df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))
df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))

# 6. Pixel spacing from filepath (float in mm)
df["px_spacing_h"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[3]))
df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))

df1 = df[df.index % 3 == 0]
df2 = df[df.index % 3 == 1]
df3 = df[df.index % 3 == 2]
df = df1.copy()
df.pop('class')
gc.collect()

del x, df1, df2, df3, _tmp_merge_df
gc.collect()
df = df.reset_index(drop=True)

print (df.shape)
if DEBUG:
    df = df.loc[(df['case_id_str'] == df['case_id_str'].values[0]) & (df['day_num'] == df['day_num'].values[0])].reset_index(drop=True)
print (df.shape)

(38496, 12)
(144, 12)


In [5]:
def rle_encode(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated

    ref: https://www.kaggle.com/stainsby/fast-tested-rle
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

PREDICTION_CLASSES = ["large_bowel", "small_bowel", "stomach"]


class SimpleInferencer(object):
    def __init__(
        self, models: List[nn.Module], threshold: float = 0.5, tta: bool = False
    ):
        self.models = models if not tta else self.make_tta_models(models)
        self.threshold = threshold
        self.tta = tta

    def make_tta_models(self, models):
        return [
            GeneralizedTTA(
                model, augment_fn=d4_image_augment, deaugment_fn=d4_image_deaugment
            )
            for model in models
        ]

    def infer_on_dataset(self, test_dataset: Dataset) -> pd.DataFrame:
        ids = []
        classes = []
        rles = []
        for idx in tqdm(range(len(test_dataset))):
            with torch.no_grad():
                # inference per individual image
                image = test_dataset[idx]["image"][None, ...].cuda()
                output = [model(image) for model in self.models]
                output = torch.stack(output)
                output = output.mean(dim=0)
                output = torch.nn.Sigmoid()(output).cpu().numpy()
            output = (output >= self.threshold).astype("uint8")
            
            root_shape = (
                test_dataset.data.iloc[idx]["slice_h"],
                test_dataset.data.iloc[idx]["slice_w"],
            )
            # output = output[0, :, :root_shape[0], :root_shape[1]]
            output = output[0, :, :root_shape[1], :root_shape[0]]
            encoded_masks = []
            for cls_idx in range(len(PREDICTION_CLASSES)):
                pred_arr = output[cls_idx]
                rle_str = rle_encode(pred_arr)
                encoded_masks.append(rle_str)
            ids.extend([test_dataset.data.iloc[idx]["id"]] * 3)
            classes.extend(PREDICTION_CLASSES)
            rles.extend(encoded_masks)

        df = pd.DataFrame()
        df["id"] = ids
        df["class"] = classes
        df["predicted"] = rles
        return df
    
class GITractDatasetTest(Dataset):
    def __init__(
        self, 
        data, 
        transform=A.Compose([A.Resize(256, 256)]),
        apply_filters: bool = False,
        use_pseudo_3d: bool = False,
        channels: Optional[int] = None,
        stride: Optional[int] = None,
    ):
        self.data = data
        self.image_path = data["f_path"]
        self.transform = transform
        self.apply_filters = apply_filters
        self.use_pseudo_3d = use_pseudo_3d
        self.channels = channels
        self.stride = stride
        if self.use_pseudo_3d:
            for i in range(self.channels):
                self.data[f"image_path_{i:02}"] = (
                    self.data.groupby(["case_id", "day_num"])["f_path"]
                    .shift(-i * self.stride)
                    .fillna(method="ffill")
                )
            self.data["image_paths"] = self.data[
                [f"image_path_{i:02d}" for i in range(self.channels)]
            ].values.tolist()
            self.data = self.data.drop(
                columns=[f"image_path_{i:02d}" for i in range(self.channels)]
            )

    def __len__(self):
        return len(self.data)

    def _load_image(self, path):
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        img = img.astype("float32")  # original is uint16
        min_val = img.min()
        max_val = img.max()
        img = img / img.max()  # scale image to [0, 1]
        if self.apply_filters:
            img_sobel = sobel(img)
            img_meijering = meijering(img)
            img = np.stack([img, img_sobel, img_meijering], -1)
        else:
            img = np.expand_dims(img, -1)
        return img

    
    def _load_images(self, paths):
        img = [
            self._load_image(path) for path in paths
        ]
        img = np.concatenate(img, -1)
        return img
    
    
    def __getitem__(self, idx):
        if self.use_pseudo_3d:
            image_paths = self.data.image_paths.values[idx]
            image = self._load_images(image_paths)
        else:
            image_path = self.data.f_path.values[idx]
            image = self._load_image(image_path)
        augmented = self.transform(image=image)
        image = augmented["image"]
        image = image.transpose(2, 0, 1)
        image = np.ascontiguousarray(image)
        return {"image": torch.tensor(image, dtype=torch.float)}

In [6]:
models = []
for fold_idx in range(4):
    model = torch.load(f'../input/gi-tract-models/steroids_v1_fold{fold_idx}.pt', map_location='cpu')
    model.eval()
    model.cuda();
    models.append(model)

In [7]:
test_set = GITractDatasetTest(
    df.reset_index(drop=True),
    use_pseudo_3d=True, 
    channels=3,
    stride=2,
    transform=A.Compose([A.PadIfNeeded(320, 384, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, position="top_left")])
)
inferencer = SimpleInferencer(models, 0.5, tta=False)

In [8]:
pred_df = inferencer.infer_on_dataset(test_set)
pred_df.head()

100%|██████████| 144/144 [00:17<00:00,  8.15it/s]


,id,class,predicted
0,case123_day20_slice_0001,large_bowel,
1,case123_day20_slice_0001,small_bowel,
2,case123_day20_slice_0001,stomach,
3,case123_day20_slice_0002,large_bowel,
4,case123_day20_slice_0002,small_bowel,


In [9]:
pred_df.loc[pred_df['predicted']!='', :].head()

,id,class,predicted
194,case123_day20_slice_0065,stomach,28624 3 28889 6 29154 8 29420 8 29686 9 29951 ...
197,case123_day20_slice_0066,stomach,27293 7 27558 9 27823 11 28088 13 28353 15 286...
200,case123_day20_slice_0067,stomach,15588 6 15853 8 16119 8 16384 10 16650 10 1691...
203,case123_day20_slice_0068,stomach,14789 7 15054 9 15319 12 15584 13 15850 14 161...
206,case123_day20_slice_0069,stomach,14259 3 14522 9 14787 12 15052 14 15318 15 155...


In [10]:
pred_df.to_csv("submission.csv", index=False)